In [1]:
!pip install -q -U bitsandbytes
!pip install -q datasets
!pip install accelerate -U
!pip install evaluate
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00


In [6]:
! pip install snorkel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.2 MB/s eta 0:00:00


In [11]:
! pip install nltk

In [16]:
! pip install utils

In [2]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback, AutoModelForCausalLM, AutoModelForSeq2SeqLM,BitsAndBytesConfig, DataCollatorWithPadding
from datasets import load_dataset, concatenate_datasets, load_from_disk, load_metric, Dataset, ClassLabel
import numpy as np
import pandas as pd

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [4]:
# Load data
df_total = pd.read_parquet('/content/drive/MyDrive/Dissertation/Data/df.parquet')
# One-shot sample
df_label_unique_sample = df_total.groupby('label_cat', group_keys=False).apply(lambda df: df.sample(1))
# Create label mapping
label = list(df_label_unique_sample['label'])
id = list(df_label_unique_sample['label_cat'])
label_to_id = dict(zip(label,id))
id_to_label = dict(zip(id,label))
# Define mapping
Mapping = ClassLabel(names= label)

# Load dataset
# Load the parquet dataset
dataset = load_dataset("parquet", data_files={'train': '/content/drive/MyDrive/Dissertation/Data/train_df.parquet', 'test': '/content/drive/MyDrive/Dissertation/Data/test_df.parquet'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-c72183ae075bf112/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
#Rename columns
df_total.rename(columns={'processed_text':'text'},inplace=True)

In [5]:
# Show without truncation
import pandas as pd
DISPLAY_ALL_TEXT = True
pd.set_option("display.max_colwidth", 0 if DISPLAY_ALL_TEXT else 50)

# Write TF function

In [ ]:
# REF: https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/02_spam_data_augmentation_tutorial.ipynb

In [10]:
from snorkel.preprocess.nlp import SpacyPreprocessor
import nltk
from nltk.corpus import wordnet as wn
from collections import OrderedDict
from snorkel.augmentation import transformation_function
nltk.download("wordnet")
spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

In [23]:
from snorkel.augmentation import transformation_function

In [32]:
# replace entity name

def get_synonym(word, pos=None):
    """Get synonym for word given its part-of-speech (pos)."""
    synsets = wn.synsets(word, pos=pos)
    # Return None if wordnet has no synsets (synonym sets) for this word and pos.
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        if words[0].lower() != word.lower():  # Skip if synonym is same as word.
            return words[0].replace("_", " ")

def replace_token(spacy_doc, idx, replacement):
    """Replace token in position idx with replacement."""
    return " ".join([spacy_doc[:idx].text, replacement, spacy_doc[1 + idx :].text])

@transformation_function(pre=[spacy])
def replace_noun_with_synonym(x):
    # Get indices of noun tokens in sentence.
    noun_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "NOUN"]
    if noun_idxs:
        # Pick random noun idx to replace.
        idx = np.random.choice(noun_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="n")
        # If there's a valid noun synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x

@transformation_function(pre=[spacy])
def replace_verb_with_synonym(x):
    # Get indices of verb tokens in sentence.
    verb_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "VERB"]
    if verb_idxs:
        # Pick random verb idx to replace.
        idx = np.random.choice(verb_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="v")
        # If there's a valid verb synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x
@transformation_function(pre=[spacy])
def replace_adjective_with_synonym(x):
    # Get indices of adjective tokens in sentence.
    adjective_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "ADJ"]
    if adjective_idxs:
        # Pick random adjective idx to replace.
        idx = np.random.choice(adjective_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="a")
        # If there's a valid adjective synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x

In [33]:
tfs = [
    replace_verb_with_synonym,
    replace_adjective_with_synonym,
    replace_noun_with_synonym
]

In [34]:
def preview_tfs(df, tfs):
    transformed_examples = []
    for f in tfs:
        for i, row in df.sample(frac=1, random_state=2).iterrows():
            transformed_or_none = f(row)
            # If TF returned a transformed example, record it in dict and move to next TF.
            if transformed_or_none is not None:
                transformed_examples.append(
                    OrderedDict(
                        {
                            "TF Name": f.name,
                            "Original Text": row.text,
                            "Transformed Text": transformed_or_none.text,
                        }
                    )
                )
                break
    return pd.DataFrame(transformed_examples)

In [35]:
preview_tfs(df_total, tfs)

,TF Name,Original Text,Transformed Text
0,replace_verb_with_synonym,redecorate interior demised premise good workmanlike manner using material good quality often may reasonable opinion landlord,redecorate interior demised premise good workmanlike manner use material good quality often may reasonable opinion landlord
1,replace_adjective_with_synonym,perpetuity period `` mean period eighty year date,perpetuity period `` average period eighty year date
2,replace_noun_with_synonym,perpetuity period `` mean period eighty year date,perpetuity time period `` mean period eighty year date


# Apply to daataset

In [56]:
from snorkel.augmentation import MeanFieldPolicy
def mean_policy(N_sample):
  mean_field_policy = MeanFieldPolicy(
      len(tfs),
      sequence_length=2,
      n_per_original=N_sample,
      keep_original=True,
      p=[0.4,0.2,0.4], # Distribution of TFs. Substitute of noun and verb is more important than adj
  )
  return mean_field_policy

In [ ]:
from transformers.models.t5.modeling_tf_t5 import TF_T5_PRETRAINED_MODEL_ARCHIVE_LIST
import datasets
train_ds = train_ds.to_pandas()

In [59]:
from snorkel.augmentation import PandasTFApplier
def apply_to_dataset(N_sample):
  tf_applier = PandasTFApplier(tfs, mean_policy(N_sample))
  df_train_augmented = tf_applier.apply(train_ds)
  Y_train_augmented = df_train_augmented["labels"].values
  return df_train_augmented
  # Concatenate X and Y variable

In [62]:
df_train_augmented_200 = apply_to_dataset(2)
df_train_augmented_300 = apply_to_dataset(3)
df_train_augmented_400 = apply_to_dataset(4)

100%|██████████| 261/261 [00:00<00:00, 325.50it/s]


In [66]:
# Save to disk
def save_to_disk(dataframe):
  text = dataframe['text']
  label = dataframe['labels']
  synthetic_dataset = Dataset.from_dict({'text': text, 'labels': label})
  return synthetic_dataset

In [67]:
output_path = '/content/drive/MyDrive/Dissertation/GPT_data/Snorkel_data'
save_to_disk(df_train_augmented_200).save_to_disk(output_path + '/' + '200')
save_to_disk(df_train_augmented_300).save_to_disk(output_path + '/' + '300')
save_to_disk(df_train_augmented_400).save_to_disk(output_path + '/' + '400')

Saving the dataset (0/1 shards):   0%|          | 0/477 [00:00<?, ? examples/s]